<a href="https://colab.research.google.com/github/komo135/forex-prediction/blob/master/ipynb/rl/dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np

In [0]:
%cd /content/drive/My Drive

try:
  import imp
  imp.reload(dqn)
except:
  import dqn

agent = dqn.Agent(restore=True, n=4, lr=1e-3)

/content/drive/My Drive


In [0]:
%cd /content

agent.run(train=True)

In [0]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# agent.test(spread=10, pip_cost=1000, los_cut=150,test_data=not True)
for _ in range(1):
  tree_idx, replay = agent.memory.sample(128)

  states = np.array([a[0][0] for a in replay], np.float32)
  new_states = np.array([a[0][3] for a in replay], np.float32)
  actions = np.array([a[0][1] for a in replay])
  rewards = np.array([a[0][2] for a in replay], np.float32).reshape((-1, 1))

  with tf.GradientTape() as tape:
    q = agent.model(states)
    target_q = agent.target_model(new_states).numpy()
    arg_q = agent.model(new_states).numpy()
    random = np.random.rand(actions.shape[0])
    arg_q = np.argmax(arg_q, -1)
    # arg_q = np.array([np.argmax(arg_q[i]) if random[i] > 0.1 else np.random.randint(arg_q.shape[1]) for i in range(arg_q.shape[0])])

    q_backup = q.numpy()

    for i in range(len(rewards)):
        # q_backup[i, actions[i]] = rewards[i] if I < 1010 and not self.restore else rewards[i] + 0.2 * target_q[i, np.argmax(arg_q[i])]
        q_backup[i, actions[i]] = rewards[i] + 0.1 * target_q[i, arg_q[i]]
    mse = tf.reduce_mean(tf.reduce_sum(tf.abs(q_backup - q) ** 1.5, -1))

  ae = np.array([sum(i) for i in np.abs(q_backup - q.numpy())])
  agent.memory.batch_update(tree_idx, ae)

  gradients = tape.gradient(mse, agent.model.trainable_variables)
  # gradients = [(tf.clip_by_value(grad, -10.0, 10.0))
  #                                   for grad in gradients]
  # agent.model.optimizer.apply_gradients(zip(gradients,agent.model.trainable_variables))
  # print(np.mean(ae))
  # print(q[0:5])
  print(mse)
print(q)
# print(np.mean(rewards)))

In [0]:
os.cpu_count()

In [0]:
m = model((224,224,3), 100)